# HoMM III moster duel simulator

This is a tool for measuring combat performance of creatures in HoMM III. The in-game AI Values are a good estimation of the unit's real strength, but they are global, which means there is only a single value to describe the quality of creature $A$ against both creatures $B$ and $C$, which can be vastly different. For example, when a Mighty Gorgon takes on a stack of Pixies, its death stare ability is almost useless, because a Pixie has a very low HP. However, in a fight between Mighty Gorgons and Azure Dragons, the Gorgons' special power is invaluable. This can't be captured by assigning only one fixed number to each monster. Mighty Gorgons vs Azure Dragons is probably the best example, but there are many more like it, albeit less extreme.

I made a simplified (but equivalent) version of Heroes III combat engine, which allowed me to do fast duel simulations between every single pair of units. That gave me lot of numbers describing how any given unit performs in a direct confrontation against every other unit in HoMM III.

Some caveats:
* it's a duel: a single stack of unit $A$ fights vs a single stack of unit $B$
* no heroes or terrain modifiers, units use their base stats
* not every special ability got implemented, but a vast majority of them did, see the Combat Rules section
* there are no obstacles on the battlefield

The Python code is available in a public repository at https://github.com/maciek16180/h3-fight-sim.

Check out < insert-address-here > for a live demonstration and results!

## Estimation method

Effectiveness of creature $A$ in combat versus creature $B$ is calculated by simulating a large number of fights between those units. The aim is to find the stack sizes for $A$ and $B$ such that both stacks have more or less equal chances against each other. The exact method is as follows:

   - $A$, $B \leftarrow$ types of fighting creatures
   
   
   - $S_A$, $S_B \leftarrow$ stacks with some starting counts
   
   
   - simulate combat between $S_A$ and $S_B$, find $S_w$, $S_l$ - winning stack and losing stack (*)
   
   
   - $\mathrm{low} \leftarrow \mathrm{count}(S_l)$
   
   
   - while $S_l$ loses to $S_w$:
       * $\mathrm{low} \leftarrow \mathrm{count}(S_l)$
       * incease $\mathrm{count}(S_l)$ by some amount (I chose $10 \%$ of the starting stack size)
       
       
   - search in $(\mathrm{low}, \mathrm{count}(S_l))$ interval for a number $k$ of creatures in $S_l$, such that the result of $S_l$ vs $S_w$ is balanced
   
   
   - set $\mathrm{count}(S_l)$ to $k$
   
   
   - return $\dfrac{\mathrm{count}(S_A)}{\mathrm{count}(S_B)}$
   
   (*) If, for example, $S_l$ is $S_A$, then by changing $S_l$ later, we also change $S_A$.
   
Whenever I simulate combat between two stacks, I actually do $n$ fights (500 by default) and count wins of each side. A stack loses, if it won less than half of the fights. Combat result is "balanced" when the difference between numbers of wins is lower than 10% of $n$.

We also have to somehow set the starting sizes for both stacks. I used AI Value for that. In a fight between $A$ and $B$ I set $\mathrm{cout}(S_A)$ = $p \times \mathrm{AI\_Value}(B)$ and $\mathrm{cout}(S_B)$ = $p \times \mathrm{AI\_Value}(A)$, with $p=10$.

I didn't do mirror fights (same unit on both sides), the result is set to tie in those cases.

### Example: Mighty Gorgon vs Scorpicore
    
    n = 500
    
    Mighty Gorgon's AI Value = 1028
    Scorpicore's    AI Value = 1589
    
    Starting counts: 15890, 10280
    
    Gorgons win 479 fights, Scorpicores' stack size up to 11308
    Gorgons win 457 fights, Scorpicores' stack size up to 12336
    Gorgons win 395 fights, Scorpicores' stack size up to 13364
    Gorgons win 393 fights, Scorpicores' stack size up to 14392
    Gorgons win 335 fights, Scorpicores' stack size up to 15420
    Gorgons win 225 fights, losing the combat

    Now binary search in (14392, 15420) for a balanced count.
    
    Check 14906: Gorgons win 298 fights, go higher
    Check 15163: Gorgons win 281 fights, go higher
    Check 15292: Gorgons win 245 fights, balanced
            
    Result is 15890 / 15292 = 1.039

This estimation took 1.4s on a middle-class laptop.
    
We need approximately 1.039 Mighty Gorgons to match a Scorpicore. The result may slightly vary, because this particular pair has a lot of randomness to it (death stare and paralysis). For n=500 I was getting results between 1.029 and 1.039. Setting n=5000 decreased variance, and the result was almost always 1.039. It upped the run time to 13.5s though. I think n=500 is both fast and consistent enough. Additionaly we see that AI underestimates Mighty Gorgon quite a bit against Scorpicore.

## Unit value

Given a subset of monsters $S$ and a creature $A$ we can try to calculate something similar to AI Value, but representing the effectiveness of $A$ specifically against $S$. This gives us the possibility to measure the quality of a certain unit against only units of higher level, or against one specific town. That quality can vary dramatically, so it's an interesting experiment. At first, I used the following formula:

$P$ - Pikeman

$A_B$ - coefficient for a pair $(A, B)$; how many $A$ is needed to match one $B$

$\mathrm{Value}(A, S) = \dfrac{\sum\limits_{s \in S} A_s^{-1}}{\sum\limits_{s \in S} P_s^{-1}} \times 80$

Some explanation: the higher $A_B$, the worse $A$'s chances in a duel with $B$. We want the unit value to behave the other way around, so we invert the results (hence the $^{-1}$). Inverted scores of $A$ are summed over each unit in $S$. To make those values meaningful, we need some point of reference, so we scale them in such a way, to make Pikeman's value always be 80 (equal to it's AI Value).

After a while I thought of another way of calculating those values. Instead of comparing $A$ to a Pikeman only during normalization, we can do it for every unit in $S$ separately:

$\mathrm{ValueAlt}(A, S) = \dfrac{80}{\left\vert{S}\right\vert} \sum\limits_{s \in S} \dfrac{P_s}{A_s}$

The second formula behaves slightly different. To show that, let's consider this simple scenario:

    A  = Marksman
    S0 = {Archer}
    S  = {Archer, Archangel}

    The table below shows combat results for Pikeman and Marksman against S.

    Unit        vs Archer   vs Archangel
    Pikeman       1.575        64.529
    Marksman	  0.641        77.595
    
    Both formulas give the same result for a set with only one element. For example, 
    considering only Archer we get:
    
    Value(A, S0) = ValueAlt(A, S0) = 1.575 / 0.641 * 80 ~= 196
    
    Now, let's add Archangel to the mix.
    
    Value(A, S)    = (1 / 0.641 + 1 / 77.595) / (1 / 1.575 + 1 / 64.529) * 80 ~= 193
    ValueAlt(A, S) = (80 / 2) * (1.575 / 0.641 + 64.529 / 77.595)             ~= 131
    
The results differ quite a bit. Adding the Archangel (against which the performance of both Pikeman and Marksman is similarily poor), strongly decreased the Marksman's $\mathrm{ValueAlt}$, while leaving its $\mathrm{Value}$ almost untouched. Both behaviors make sense in their own way, and I am not sure which one I like better. I include the alternative version as a checkbox on site.

## Combat simulation

*In this section, a *shooter* means a creature with ranged attack (i.e. has number of shots > 0), and a *walker* is a creature without one.*

The combat engine I use is greatly simplified, but in a vast majority of cases should be equivalent to the original one. I am only interested in battles between singular stacks (like 100 Archers vs 20 Crusaders), so there is no need to implement the entire battlefield. This allowed me to describe the flow of combat using very simple rules. There is no explicit stack movement or combat rounds. Defending does not exist; it can prolong the combat indefinitely and we can't have that. As far as waiting goes, there is no point to do it outside of the walker - shooter encounter described below.

I distinguish three basic cases:

    1) walker - walker
    2) shooter - shooter
    3) walker - shooter
    
**1) Walker - walker**

Simple case, the stacks keep attacking each other until one of them is dead. The stack with the higher speed starts, ties are broken at random.

**2) Shooter - shooter**

THe stacks keep shooting at each other until one of them is dead or one of them has no more ammunition. In the latter case we are reduced to the case (3). The order of attacks is the same as in case (1).

**3) Walker - shooter**

The most complicated case. In short: shooter shoots at walker until walker catches it, then they fight hand-to-hand. I calculate how many full-strength shots walker can avoid if it's smart.

Long version (Python-ish syntax):

    d  = number of hexes the walker has to travel to be able to attack
    ws = walker's speed
    
    if walker is slower than shooter (*):
        number_of_shots = (d // ws + (d % ws > 0))

        A walker can avoid at most one full shot. 
        To do so, it has to not enter the shooter's range when doing it's first move.
        Shooter is faster, so it can wait to force walker to move first, 
        which makes walker unable to avoid a full shot by waiting. 
        No creature in the game is slow enough to be outside of range after two rounds.
        
        Optimal length of the first move for a walker is m, calculated as follows:
            m = d % ws
            if m == 0:
                m = ws
        Every subsequent move has length ws. Making first move longer than m is
        unnecessary, making it any shorter increases the number of rounds without attack 
        for a walker.
        If shooter has no penalty after walker marches m hexes forward, than the walker
        can't profitably avoid any full shots. Otherwise, exactly one shot is fired with
        penalty. 
          
    if walker is faster than shooter (**):
        numer_of_shots = (d // ws - (d % ps == 0))

        In this case walker can avoid one additional shot, because it can wait to force
        shooter to shoot first. Up to one shot is avoided by smart movement, like in 
        the previous case. Overall, walker can avoid one or two full shots, depending on
        its speed.
      
    if walker and shooter have the same speed:
        In this case starting stack is chosen randomly. Waiting is not beneficial 
        anymore, because it changes the turn order for the rest of the fight. This makes
        walker unable to avoid any shots by waiting.
        
        number_of_shots is the same as in (*) if the shooter starts, and the same as in
        (**) otherwise.
        
    Now shooter shoots at walker number_of_shots times, some of which are penalized
    accordingly. After that we have case (1), with the walker being starting stack.

## Combat rules

I took into account every basic combat mechanic and creature characteristic, like attack, defense, number of shots, retaliation, creature size, etc. In addition to that, I implemented most special abilities:

  - no enemy retaliation
  - no meele penalty
  - double attack or shot
  - rebirth (Phoenix)
  - enemy defense reduction (Behemoth, Ancient Behemoth)
  - death blow (Dread Knight)
  - life drain (Vampire Lord)
  - death stare (Mighty Gorgon)
  - hate (e.g. Angel - Devil) and double damage (opposing Elementals)
  - fear (Azure Dragon)
  - regeneration (Wight, Wraith, Troll)
  - fire shield (Efreet Sultan)
  - acid breath (Rust Dragon)
  - lighning strike (Thunderbird)
  - aging (Ghost Dragon)
  - poison (Wyvern Monarch)
  - curse (Mummy, Black Knight, Dread Knight) (small discrepancy in duration - sometimes ends half a round too early)
  - weakness (Dragon Fly) (same as above)
  - disease (Zombie) (same as above)
  - blind (Unicorn, War Unicorn)
  - paralyzing venom (Scorpicore)
  - petrification (all Basilisks and Medusas)
  
In all cases (I hope) I consider the following creatures' immunities:

  - golems - less magic damage (%)
  - Firebird, Phoenix, Efreet, Efreet Sultan - immune to fire spells
  - all undeads - resistances of undeads
  - all elementals - resistances of elementals
  - gargoyles, golems - resistances of non-living creatures
  - Green Dragon, Red Dragon, Azure Dragon - immune to spells 1-3
  - Gold Dragon - immune to spells 1-4
  - Black Dragon, Magic Elemental - immune to all spells
  - Dwarf, Battle Dwarf, Crystal Dragon - magic resistance (%)
  - Giant, Titan - immune to mind spells
  - Troglodytes, Infernal Troglodytes - immune to blind and petrification
  
Not implemented:

  - any spells cast by creatures (most of them are useless in a duel anyway, others can take too much time to implement; this does not include incidental effects, like Efreet's Fire Shield or Dragon Fly's Weakness)
  - jousting (Cavalier, Champion)
  - strike and return (Harpy Hag)
  - abilities that are useless in a duel (like Dendroids' binding)
  - everything else I forgot

## Various stuff

The calculated relative strength values are stable only for large stacks. For example, when Pikemen deal 700 damage, they kill 2 Archangels. When they deal 10 times more, they kill 28 Archangels, which is 14 times more. This leads to the situation where 8 Archangels almost always win against 520 Pikemen, but when you multiply both stack sizes by 10, suddenly the result is not far from 50-50. It's caused by the fact, that low amouts of damage have no effect on combat abilities of high-level units (the stack's efficiency stays the same until some units actually get killed). It means that for small stack sizes, calculated values are going to favor high-level units. You can of course run the simulator yourself to find the matching Pikemen stack for any number of Archangels.

*-*

The entire thing is written in Python 3.6.3 and for the most part doesn't require any non-standard modules. I had to make small concessions to be compatible with Transcrypt (http://www.transcrypt.org/), which I use to automagically get the Javascript version running on site.

*-*

Useful links:

http://heroes.thelazy.net/ - invaluable source of knowledge about Heroes III

http://heroescommunity.com/viewthread.php3?TID=19321&pagenumber=2 - description of CRTRAITS.TXT

http://heroescommunity.com/viewthread.php3?TID=12210&pagenumber=2 - explanation of Mighty Gorgon's death stare